In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def crawl_pnj_gold_price():
    # URL của trang web PNJ
    url = "https://www.pnj.com.vn/blog/gia-vang/"
    
    # Headers để tránh bị chặn
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        # Gửi request tới website
        print("Đang gửi request tới", url)
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        print("Request thành công, đang phân tích HTML...")
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Tìm các phần tử có thể chứa giá vàng (div, span, hoặc table)
        price_containers = soup.find_all(['table', 'div', 'span'], class_=['price', 'gold', 'table'])
        if not price_containers:
            print("Không tìm thấy phần tử nào liên quan đến giá vàng!")
            print("Một phần HTML mẫu:", soup.prettify()[:1000])  # In nhiều hơn để kiểm tra
            return None
        
        print(f"Tìm thấy {len(price_containers)} phần tử tiềm năng trong trang web")
        
        # Danh sách để lưu dữ liệu
        data = []
        
        # Lấy thời gian crawl
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Kiểm tra nội dung từng phần tử
        for container in price_containers:
            text = container.get_text(strip=True)
            if "mua" in text.lower() or "bán" in text.lower():  # Tìm từ khóa liên quan đến giá
                # Giả định cấu trúc: loại vàng - giá mua - giá bán
                rows = container.find_all(['tr', 'div'])
                for row in rows:
                    cols = row.find_all(['td', 'span', 'div'])
                    if len(cols) >= 2:  # Cần ít nhất 2 cột (mua và bán)
                        location_or_type = cols[0].text.strip()
                        buy_price = cols[1].text.strip() if len(cols) > 1 else "N/A"
                        sell_price = cols[2].text.strip() if len(cols) > 2 else "N/A"
                        
                        data.append({
                            'timestamp': timestamp,
                            'location': location_or_type,
                            'buy_price': buy_price,
                            'sell_price': sell_price
                        })
        
        # Kiểm tra và in dữ liệu
        if not data:
            print("Không tìm thấy dữ liệu giá vàng cụ thể!")
            print("Một phần HTML mẫu:", soup.prettify()[:1000])
            return None
        
        print("\nDữ liệu giá vàng PNJ vừa crawl:")
        print("-" * 50)
        for item in data:
            print(f"Thời điểm: {item['timestamp']}")
            print(f"Nơi giao dịch/Loại vàng: {item['location']}")
            print(f"Giá mua vào: {item['buy_price']}")
            print(f"Giá bán ra: {item['sell_price']}")
            print("-" * 50)
        
        # Chuyển thành DataFrame và lưu file
        df = pd.DataFrame(data)
        df.to_csv('pnj_gold_prices.csv', index=False, encoding='utf-8-sig')
        print("\nĐã lưu dữ liệu vào file pnj_gold_prices.csv")
        
        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi gửi request: {str(e)}")
        return None
    except Exception as e:
        print(f"Lỗi khác khi crawl dữ liệu: {str(e)}")
        return None

if __name__ == "__main__":
    crawl_pnj_gold_price()

Đang gửi request tới https://www.pnj.com.vn/blog/gia-vang/
Request thành công, đang phân tích HTML...
Không tìm thấy phần tử nào liên quan đến giá vàng!
Một phần HTML mẫu: <!DOCTYPE html>
<html dir="ltr" lang="vi">
 <head>
  <meta charset="utf-8"/>
  <title>
   Giá Vàng Hôm Nay: PNJ, SJC, 9999, 18K, 24K Cập Nhật Mới Nhất
  </title>
  <meta content="index,follow" name="robots"/>
  <meta content="index,follow" name="googlebot"/>
  <meta content="Cập Nhật Mới Nhất Bảng Giá Vàng Hôm Nay: PNJ, SJC, Vàng 9999, 18K, 24K ❤️Giá Miếng Vàng ⭐️Trong Nước ✔️Giá Vàng Thế Giới ☎️1800545457 (Miễn Phí)" name="description"/>
  <meta content="PNJ" name="twitter:site"/>
  <meta content="website" property="og:type"/>
  <meta content="Giá Vàng Hôm Nay: PNJ, SJC, 9999, 18K, 24K Cập Nhật Mới Nhất" property="og:title"/>
  <meta content="Cập Nhật Mới Nhất Bảng Giá Vàng Hôm Nay: PNJ, SJC, Vàng 9999, 18K, 24K ❤️Giá Miếng Vàng ⭐️Trong Nước ✔️Giá Vàng Thế Giới ☎️1800545457 (Miễn Phí)" property="og:description"/>
  